# PyNodeWidget Demo (NEW API)

This notebook demonstrates the new class-based API for PyNodeWidget using Pydantic models.

## Key Features:
- **Class-based node definitions** - Inherit from `JsonSchemaNodeWidget`
- **Pydantic models** for type-safe parameters
- **Auto-generated JSON Schema** from Pydantic
- **Automatic registration** with `NodeFlowWidget(nodes=[...])`

In [1]:
from pydantic import BaseModel, Field
from pynodewidget import NodeFlowWidget, JsonSchemaNodeWidget


# Define parameter models using Pydantic
class ProcessorParams(BaseModel):
    """Parameters for data processor node."""
    name: str = Field(default="processor", description="Processor name")
    count: int = Field(default=10, ge=1, le=100, description="Number of items to process")
    enabled: bool = Field(default=True, description="Enable processing")


class FilterParams(BaseModel):
    """Parameters for filter node."""
    threshold: float = Field(default=0.5, ge=0, le=1, description="Filter threshold")
    filter_type: str = Field(default="low", description="Type of filter to apply")


class OutputParams(BaseModel):
    """Parameters for output node."""
    format: str = Field(default="json", description="Output format")


# Define node classes using the new API
class ProcessorNode(JsonSchemaNodeWidget):
    """Process data with configurable parameters."""
    label = "Data Processor"
    parameters = ProcessorParams
    icon = "⚙️"
    description = "Process data with configurable parameters"
    inputs = [{"id": "input", "label": "Input"}]
    outputs = [{"id": "output", "label": "Output"}]


class FilterNode(JsonSchemaNodeWidget):
    """Filter data based on threshold."""
    label = "Filter Node"
    parameters = FilterParams
    icon = "🔍"
    description = "Filter data based on threshold"
    inputs = [{"id": "in", "label": "Input"}]
    outputs = [{"id": "out", "label": "Output"}]


class OutputNode(JsonSchemaNodeWidget):
    """Display or export results."""
    label = "Output Node"
    parameters = OutputParams
    icon = "📤"
    description = "Display or export results"
    inputs = [{"id": "data", "label": "Data"}]


# Create widget with automatic node registration
widget = NodeFlowWidget(
    nodes=[ProcessorNode, FilterNode, OutputNode],
    height="600px"
)

# Optionally add some initial nodes to the canvas
widget.nodes = [
    {
        "id": "1",
        "type": "processor_node",  # Auto-converted from ProcessorNode
        "position": {"x": 50, "y": 50},
        "data": {
            "label": "Data Processor",
            "parameters": widget.node_templates[0]["defaultData"]["parameters"],
            "inputs": [{"id": "input", "label": "Input"}],
            "outputs": [{"id": "output", "label": "Output"}],
            "values": {}
        }
    }
]

widget

## Get the current flow data

After modifying the graph in the UI, run this cell to see the current state:

In [2]:
# Get current flow data
flow_data = widget.get_flow_data()
print("Nodes:", len(flow_data["nodes"]))
print("Edges:", len(flow_data["edges"]))
flow_data

Nodes: 1
Edges: 0


{'nodes': [{'id': '1',
   'type': 'processor_node',
   'position': {'x': 50, 'y': 50},
   'data': {'label': 'Data Processor',
    'parameters': {'description': 'Parameters for data processor node.',
     'properties': {'name': {'default': 'processor',
       'description': 'Processor name',
       'title': 'Name',
       'type': 'string'},
      'count': {'default': 10,
       'description': 'Number of items to process',
       'maximum': 100,
       'minimum': 1,
       'title': 'Count',
       'type': 'integer'},
      'enabled': {'default': True,
       'description': 'Enable processing',
       'title': 'Enabled',
       'type': 'boolean'}},
     'title': 'ProcessorParams',
     'type': 'object'},
    'inputs': [{'id': 'input', 'label': 'Input'}],
    'outputs': [{'id': 'output', 'label': 'Output'}],
    'values': {}}}],
 'edges': []}

## Access individual nodes and edges

In [3]:
# Access nodes and edges directly
print(f"Number of node types registered: {len(widget.node_templates)}")
print("\nAll nodes:")
for node in widget.nodes:
    print(f"  - {node['id']}: {node['data'].get('label', 'No label')} (type: {node['type']})")

print("\nAll edges:")
for edge in widget.edges:
    print(f"  - {edge['id']}: {edge['source']} -> {edge['target']}")
    
print("\nRegistered node types:")
for template in widget.node_templates:
    print(f"  - {template['type']}: {template['label']}")

Number of node types registered: 3

All nodes:
  - 1: Data Processor (type: processor_node)

All edges:

Registered node types:
  - processor_node: Data Processor
  - filter_node: Filter Node
  - output_node: Output Node


## Save and load flows

In [4]:
# Save flow to JSON file
widget.export_json('my_flow.json')

✓ Flow exported to my_flow.json


'my_flow.json'

In [5]:
# Load flow from JSON file
# widget.load_json('my_flow.json')

In [6]:
# You can also add more node types dynamically
class TransformParams(BaseModel):
    """Parameters for transform node."""
    operation: str = Field(default="normalize", description="Transform operation")
    scale: float = Field(default=1.0, ge=0.1, le=10.0, description="Scale factor")

class TransformNode(JsonSchemaNodeWidget):
    """Transform data."""
    label = "Transform"
    parameters = TransformParams
    icon = "🔄"
    inputs = [{"id": "in", "label": "Input"}]
    outputs = [{"id": "out", "label": "Output"}]

# Register the new node type
widget.register_node_type(TransformNode)
print(f"✓ Now have {len(widget.node_templates)} node types registered")

✓ Now have 4 node types registered
